In [1]:
import pandas as pd
import re
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, GPTNeoForCausalLM, GPT2Tokenizer
from transformers import pipeline
from transformers import Trainer, TrainingArguments
import torch
import gc
import wandb

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Laptop GPU'

In [2]:
train_path = 'datasets/train_dataset.csv'
test_path = 'datasets/test_dataset.csv'

model_path = "./models/gpt-neo-dialogs"

In [4]:
def tokenize_function(examples):
    return tokenizer(examples['text'], return_special_tokens_mask=True)

datasets = load_dataset('csv', data_files={'train': train_path, 'test': test_path})

tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['text'],
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Using custom data configuration default-d27f0e2a7b0ecb18
Reusing dataset csv (C:\Users\zhuzi\.cache\huggingface\datasets\csv\default-d27f0e2a7b0ecb18\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2085 > 2048). Running this sequence through the model will result in indexing errors


  0%|          | 0/9 [00:00<?, ?ba/s]

In [5]:
max_seq_length = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // max_seq_length) * max_seq_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
)

  0%|          | 0/75 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [10]:
hyperparams = [
    (5e-5, 'linear', True), 
    (1e-5, 'cosine', False),
    (5e-6, 'constant', False),
    (5e-5, 'cosine', True),
    (5e-6, 'linear', False),
    (5e-5, 'linear', False),
]

for (learning_rate, lr_scheduler_type, adafactor) in hyperparams:
    model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
    tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
    tokenizer.pad_token = tokenizer.eos_token
    
    wandb.init(project='gpt-neo-dialogs', entity='ziyizhu')

    training_args = TrainingArguments(
        output_dir=model_path,         # The output directory
        overwrite_output_dir=True,     # overwrite the content of the output directory
        num_train_epochs=3,            # number of training epochs
        learning_rate=learning_rate,
        lr_scheduler_type=lr_scheduler_type,
        adafactor=adafactor,
        per_device_train_batch_size=8, # batch size for training
        per_device_eval_batch_size=16, # batch size for evaluation
        evaluation_strategy="steps",
        eval_steps=500,                # Number of update steps between two evaluations.
        save_steps=1000,               # after # steps model is saved
        warmup_steps=500,              # number of warmup steps for learning rate scheduler
        report_to="wandb",             # enable logging to W&B
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
    )
    
    torch.cuda.empty_cache()
    gc.collect()

    trainer.train()
    
    wandb.config.test_size = 0.1
    wandb.finish()

loading configuration file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json from cache at C:\Users\zhuzi/.cache\huggingface\transformers\29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
Model config GPTNeoConfig {
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",


PyTorch: setting up devices
The following columns in the training set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running training *****
  Num examples = 43768
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16413
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
500,2.826800,2.844819
1000,2.758500,2.845232
1500,2.761100,2.840645
2000,2.749400,2.840839
2500,2.735000,2.833065
3000,2.713400,2.833573
3500,2.709500,2.827738
4000,2.701600,2.825691
4500,2.686700,2.821231
5000,2.684800,2.818866


The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkpoint to ./models/gpt-neo-dialogs\checkpoint-1000
Configuration saved in ./models/gpt-neo-dialogs\checkpoint-1000\config.json
Model weights saved in ./models/gpt-neo-dialogs\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoFo

***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkpoint to ./models/gpt-neo-dialogs\checkpoint-13000
Configuration saved in ./models/gpt-neo-dialogs\checkpoint-13000\config.json
Model weights saved in ./models/gpt-neo-dialogs\checkpoint-13000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkp

...

Step,Training Loss,Validation Loss
500,2.822200,2.828465
1000,2.742500,2.824556
1500,2.742600,2.819348
2000,2.730900,2.820631
2500,2.717300,2.811087
3000,2.696400,2.810147
3500,2.692800,2.811990
4000,2.685300,2.805316
4500,2.671600,2.803370
5000,2.670400,2.801223


The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkpoint to ./models/gpt-neo-dialogs\checkpoint-1000
Configuration saved in ./models/gpt-neo-dialogs\checkpoint-1000\config.json
Model weights saved in ./models/gpt-neo-dialogs\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoFo

***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkpoint to ./models/gpt-neo-dialogs\checkpoint-13000
Configuration saved in ./models/gpt-neo-dialogs\checkpoint-13000\config.json
Model weights saved in ./models/gpt-neo-dialogs\checkpoint-13000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 4977
  Batch size = 16
Saving model checkp

eval/loss,2.81839
eval/runtime,22.5675
eval/samples_per_second,220.538
eval/steps_per_second,13.825
train/epoch,3.0
train/global_step,16413
train/learning_rate,0.0
train/loss,2.4165
train/total_flos,8574384177414144.0
train/train_loss,2.55383
train/train_runtime,3091.4822


eval/loss,█▇▆▆▄▃▄▂▂▁▃▄▃▄▃▃▃▃▃▂▂▅▇▆▇▆▆▆▆▆▅▅
eval/runtime,▄█▄▁▃▄▄▄▃▆▄▅▄▄▄▆▄▅▅▆▅▄▅▅▅▅▄▄▅▅▅▅
eval/samples_per_second,▅▁▅█▆▅▅▅▅▃▅▄▅▅▅▃▅▄▄▃▄▅▄▄▄▄▅▅▄▄▄▄
eval/steps_per_second,▅▁▅█▆▅▅▅▆▃▆▄▆▅▅▃▅▄▄▃▄▅▄▄▄▄▅▅▄▄▄▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▇▇▆▆▆▆▆▅▅▅▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
